## 레이블 없이 수집된 로그 데이터에서 의미 있는 인사이트를 얻기 위해 다음과 같은 단계로 접근

## 1. 데이터 탐색 및 시각화
- **데이터 분포 확인**: 각 변수(ex: 심박수, 체온, 속도 등)가 일정한 범위 내에서 분포하는지 확인하여 이상값을 탐지
- **시각화**: 데이터의 시계열 변화를 시각화(ex: 시간별 심박수, 속도 변화 등)해 특정 패턴이나 이상 징후를 눈으로 파악
- **상관관계 분석**: 자이로스코프와 위치 정보가 특정 신호와 어떤 상관관계를 갖는지 확인하여 변수 간 관계를 이해

## 2. 잠재적 이벤트 식별
> 레이블이 없기 때문에 비지도 학습 방법을 통해 잠재적 이벤트나 중요한 순간을 구분 가능
- **이상 탐지(Anomaly Detection)**: 자이로스코프 값, 속도, 심박수 등에서 특정 범위와 다른 값이 나타나는 구간을 이상 이벤트로 설정하여 분류
- **클러스터링**: K-Means, DBSCAN과 같은 클러스터링 기법을 사용해 비슷한 패턴을 가진 데이터 군집을 찾고, 각 클러스터가 의미하는 이벤트를 해석

## 3. 피처 엔지니어링 및 유의미한 피처 추출
> 데이터를 다양한 파생 피처로 변환해 행동이나 상태를 유추 가능
- **시간 기반 피처**: 시간대별 패턴 분석을 위해 데이터에서 하루 중 시간, 주간 패턴 등을 파생시켜 분석
- **가속도 및 회전량 계산**: 자이로스코프 데이터를 기반으로 가속도, 방향 변환 정도를 추출하여 급격한 움직임(예: 낙상 가능성)을 감지
- **GPS 이동 속도 및 거리**: 위치 데이터를 기반으로 이동 속도와 거리를 계산해 비정상적인 이동이나 특정 구간 내의 이동 패턴을 추출 가능

## 4. 모델 학습 및 레이블링 작업
> 프로젝트 목적에 맞는 위험 예측 모델을 학습하고, 데이터를 자동으로 레이블링하는 시스템을 구축 가능
- **기존 공개 데이터셋 활용**: 공개된 레이블링 데이터셋을 사용하여 유사한 상황의 행동을 예측하는 모델을 사전 학습하고, 이를 활용해 레이블이 없는 데이터를 분류
- **자체 레이블링**: 주기적으로 특정 패턴을 가진 데이터를 위험 상황으로 간주하고, 이 데이터를 기반으로 점진적으로 레이블링을 늘려가는 방법을 고려 가능

## 5. 비지도 학습 기반 위험 평가 시스템 구축
> 비지도 학습 알고리즘을 통해 레이블 없이도 위험도를 예측할 수 있는 시스템을 구축
- **Autoencoder 및 Isolation Forest**: Autoencoder를 통해 비정상적인 데이터 포인트를 감지하거나 Isolation Forest를 사용해 이상 감지를 수행하여 위험도를 판단
- **시계열 이상 탐지**: 시계열 데이터에 특화된 시계열 이상 탐지 모델을 적용해 일정 패턴을 벗어나는 구간을 감지하여 자동으로 위험을 평가

# 로그 데이터만으로 위험 예측 모델을 구현.
> 로그 데이터가 특정 상황을 나타낸느 이벤트가 아니기 때문에 데이터를 적절히 가공하고, 특징을 추출하여 잠재적 위험 요소를 유추해야 함.
> 주어진 데이터만으로 위험 예측을 구현하기 위한 방법

## 1. 로그 데이터의 이해 및 가공
- **위치 데이터 분석**: `latitude`, `longitude`를 기반으로 작업자의 움직임과 위치 변화를 추적 가능
- **속도 및 가속도 계산**: `speed` 및 `bearing` 데이터를 이용해 작업자의 이동 속도와 가속도 변화를 파악. 이러한 값이 갑작스럽게 변하거나 특정 패턴을 나타낼 경우 이를 이상 행동의 징후로 간주 가능
- **시간 변수 활용**: `unixtime` 또는 `checktime`을 활용하여 시간 기반 특징을 생성 가능. 예를 들어, 시간대별 작업 패턴을 파악하거나, 특정 시간대에 주로 발생하는 이동 패턴을 학습하여 비정상적 움직임을 감지

## 2. 피처 엔지니어링 (Feature Engineering)
> 로그 데이터에서 위험을 예측할 수 있는 의미 있는 피처를 추가로 생성해야 함
- **이동 거리와 방향 변화:**
    - 이전 위치와 현재 위치의 차이를 이용하여 이동 거리(`distance`)를 계산. 일정 거리 이상 갑작스러운 이동이 발생할 경우 이를 비정상 움직임으로 판단 가능
    - `bearing` 값을 사용해 특정 방향으로 일정하게 움직이다가 갑작스럽게 방향을 변경하는 경우, 이를 위험 행동의 징후로 볼 수 있음
- **속도와 가속도 변화량:**
    - `speed`와 이전 `speed`의 차이를 통해 가속도(`acceleration`)를 계산함. 작업자가 급격히 이동 속도를 높이거나 낮출 때, 이는 사고나 급작스러운 움직임의 위험 신호일 수 있음
    - 일정 시간 내 가속도와 속도의 변동 폭을 피처로 추가하여 위험 상황을 포착
- **정체된 상태 탐지:**
    - `speed` 값이 오랜 시간 동안 0으로 유지되면 작업자가 고정된 위치에 있는 것으로 판단 가능. 이런 상태가 예기치 않게 발생했을 경우 안전에 문제가 있을 가능성이 있음
- **시간 기반 패턴:**
    - 시간대별로 패턴을 나눠 작업자 위치나 이동 패턴이 시간대에 따라 다르게 나타나는지 파악. 예를 들어, 새벽 시간대에 급격한 이동이 발생하면 비정상 행동으로 판단 가능

## 3.비지도 학습 모델 활용
> 라벨이 없는 로그 데이터에서는 비지도 학습을 사용하여 이상 행동을 감지 가능.
- **Isolation Forest**: 이 모델은 대부분의 정상적인 데이터 포인트와 다른 데이터 포인트(이상값)를 찾는 데 효과적. 이동 거리, 속도, 가속도 등의 피처를 기반으로 이상치를 탐지하여 비정상적인 움직임을 감지
- **Autoencoder**: Autoencoder는 데이터의 일반적인 패턴을 학습한 후, 일반적인 패턴에서 벗어난 데이터를 이상값으로 간주 가능. 가속도, 속도 변동과 같은 피처를 입력으로 사용하여 비정상적 행동을 탐지함.
- **DBSCAN (밀도 기반 클러스터링)**: 밀도가 낮은 클러스터를 비정상 상태로 간주할 수 있으며, 움직임 패턴이 일정한 클러스터로 묶이지 않는 경우 위험 신호로 판단 가능

## 4. 실시간 위험 예측 및 경고 설정
> 이상값을 탐지할 경우 이를 실시간으로 알림으로 전환하여 작업자에게 위험 경고를 제공 가능-
- **특정 조건 설정**: 일정 거리 이상의 이동, 급격한 가속도 변화, 일정 속도 이하의 정체 상태와 같은 조건을 이상 행동으로 정의함
- **알림 시스템**: 특정 조건이 충족될 경우 작업자 또는 관리자에게 경고 알림을 발송하여 즉각적인 조치를 취할 수 있도록 함. 예를 들어, Slack, 이메일, 문자 메시지를 통해 알림을 보낼 수 있음.

## 5. 평가 및 개선
- **이상값에 대한 후처리**: 예측된 이상값이 실제로 위험한 상황을 반영하는지 확인하고, 필요할 경우 모델을 개선함. 예를 들어, 이상 행동이 자주 발생하는 위치에서의 작업 환경을 분석하여 데이터의 패턴을 보강 가능
- **데이터 축적 및 피드백 루프**: 이상 탐지 결과와 실제 작업 환경 데이터를 기반으로 모델을 지속적을 업데이트하여 점점 더 정확한 예측이 가능하도록 함

# 세 데이터셋(`UserVitalSign`, `UserLocationLog`, `UserGyro`)을 이용해 **위험 상황 예측 시스템**을 구현

## 1. 데이터 동기화 및 통합
- **시간 동기화**: `vitaldate` (UserVitalSign), `checktime` (UserLocationLog), `VitalDate` (UserGyro)열을 기준으로 데이터를 통합하여 같은 시간대에 수집된 정보를 하나로 결합
- **사용자별 그룹화**: `usercode`를 기준으로 사용자별로 데이터를 그룹화하고, 각 사용자의 위치, 생체 신호, 자이로스코프 데이터를 실시간으로 추적할 수 있게 준비함

## 2. 피처 엔지니어링
- **위치 데이터로부터 피처 추출:**
    - `latitude`와 `longitude`를 사용하여 작업자의 이동 패턴을 파악하고, 이동 속도(`speed`)와 가속도(`acceleration`)를 계산하여 갑작스러운 이동이 발생하는지 확인함
    - `altitude`와 `bearing` 값을 사용하여 작업자가 높은 고도에서 급격하게 이동하거나 방향을 바꾸는 경우를 탐지하여 낙상 가능성을 평가함
- **생체 신호 데이터로부터 피처 추출:**
    - `heartbeat` 값의 변동을 분석하여 심박수가 평소보다 급격히 상승하거나 비정상적인 상태가 발생할 때를 감지함.
    - `temperature`와 `outsidetemperature` 간의 차이를 계산하여 체온 이상 징후를 포착하고, 외부 환경과 체온이 비정상적으로 상이한 경우 위험으로 간주함
- **자이로스코프 데이터로부터 피처 추출:**
    - `X`, `Y`, `Z` 값으로 가속도를 계산하여 급격한 움직임이나 회전이 발생하는 순간을 탐지함. 예를 들어, 회전량이 급격히 증가하면 낙상 가능성을 높게 판단할 수 있음

## 3. 모델 학습 및 위험 상황 예측 모델 구축
- **비지도 학습 기반 이상 탐지:**
    - 데이터에 라벨이 없는 경우, `Isolation Forest`나 `DBSCAN`을 사용해 일반적이지 않은 움직임, 위치 변화 또는 생체 신호의 이상 변화를 탐지함
    - 자이로스코프 데이터의 급격한 변동, 일정하지 않은 심박수 상승 등이 감지되면 이를 위험 상태로 판단 가능
- **지도 학습 기반 위험 예측 모델:**
    - 만약 과거에 발생한 위험 상황에 대한 기록이 있는 경우, 이를 바탕으로 `LSTM`이나 `CNN-LSTM` 모델을 학습시켜 시간에 따른 변화를 학습하고 예측하도록 함.
    - 심박수 상승과 급격한 위치 변화가 동시에 발생하는 패턴을 학습하여 이를 위험으로 간주 가능

## 4. 실시간 위험 경고 시스템
- **실시간 데이터 처리**: 새로운 데이터가 들어올 때마다 처리할 수 있도록 `Kafka`와 같은 스트리밍 플랫폼을 사용해 실시간 데이터를 수집하고 모델이 예측을 실행하도록 설정함
- **경고 설정 및 알림**: 특정 조건(예: 심박수 상승과 동시에 이동 속도 급증)이 충족되면 즉시 경고 알림을 발송함
- **대시보드 및 알림 시스템**: 현장 관리자와 작업자가 즉시 확인할 수 있도록 실시간 모니터링 시스템을 구축하고, 스마트폰 앱이나 웨어러블 디바이스를 통해 알림을 보냄

## 5. 모델 평가 및 개선
- **이상 행동 기록 및 학습 데이터 업데이트**: 예측된 위험 상황이 실제 위험 상황과 일치하는지 평가하고, 필요에 따라 모델을 지속적으로 업데이트함
- **사용자 피드백 반영**: 현장 관리자의 피드백을 반영하여 특정 패턴을 위험 패턴으로 학습하거나 비정상적으로 분류되는 데이터를 추가 학습하여 정확도를 높임